# Notebook 02: Define Quality and Separate Datasets

## 🎯 What is This Notebook About?

This notebook helps you understand what makes a "good" close note and separates your incident dataset into two groups:
- **Reference Dataset** (high-quality close notes) - These will serve as examples of what good close notes look like
- **Other Incidents Dataset** - All remaining incidents (for comparison)

**Why this matters:**
- We need clear examples of good close notes to compare against
- These reference examples will be used to evaluate other close notes (existing or AI-generated)
- By separating them, we can see the difference between high-quality and standard close notes

---

## 📚 Key Concepts Explained

### What Makes a "Good" Close Note?

A good close note should:
1. **Be informative** - Contains specific details about the problem and solution
2. **Be complete** - Covers what happened, what was done, and the outcome
3. **Avoid generic phrases** - Doesn't just say "Issue resolved" without explanation
4. **Be professional** - Clear, well-structured, and easy to understand

**Example of a GOOD close note:**
> "Investigated the reported issue with Workday crashing when saving files. Cleared browser cache and cookies, updated browser to latest version. Verified user can now save files successfully. Issue resolved and confirmed with user."

**Example of a BAD/Generic close note:**
> "Issue resolved."

**Why the difference matters:**
- Good close notes help others understand what happened and how it was fixed
- Generic close notes don't provide useful information
- We'll use good examples as "reference" to evaluate other close notes

---

## 🎯 Objectives

This notebook will:
1. **Define quality criteria** - Explain what makes a close note "good"
2. **Examine quality scores** - Look at how close notes are scored
3. **Filter high-quality examples** - Identify close notes that meet our criteria
4. **Separate into two datasets:**
   - **Reference Dataset** - High-quality close notes (ground truth)
   - **Other Incidents Dataset** - Remaining incidents (for comparison)
5. **Save datasets** - Save both for use in later notebooks

---

## 📋 How We'll Separate the Data

**Separation Strategy:**
- **Reference Dataset** (`reference_close_notes.csv`): 
  - High-quality close notes that meet all criteria
  - Will be used as examples/references for evaluation
  
- **Other Incidents Dataset**:
  - All incidents that don't meet the "high-quality" criteria
  - Will be used for comparison in later notebooks

**Selection Criteria for Reference Dataset:**
- High information score (`info_score_close_notes` ≥ 0.8)
- Low generic content score (`info_score_poor_close_notes` ≤ 0.1)
- Complete and informative text (not just "Issue resolved")
- Minimum length (ensures sufficient detail)
- Diverse examples across different categories


## 1. Import Libraries and Setup

**What we're doing:** Loading the tools we need to work with data and create visualizations.

**Why:** Just like a carpenter needs a hammer and saw, we need specific tools (libraries) to work with data.

**What to expect:** You'll see a success message when everything is loaded correctly.


In [ ]:
# Import required libraries
# Think of these like tools in a toolbox - each one does a specific job

import pandas as pd  # For working with data tables (like Excel spreadsheets)
import numpy as np   # For doing math calculations
import matplotlib.pyplot as plt  # For creating charts and graphs
import seaborn as sns  # For making prettier charts
from pathlib import Path  # For handling file paths
import sys
import re  # For text pattern matching (finding generic phrases)

# Add src directory to path so we can use our helper functions
sys.path.append(str(Path("../src").resolve()))

# Import our custom helper functions
from utils import load_incident_dataset, calculate_basic_stats

# Set up plotting style (makes our charts look nicer)
try:
    plt.style.use('seaborn-v0_8')
except OSError:
    try:
        plt.style.use('seaborn')
    except OSError:
        plt.style.use('default')
sns.set_palette("husl")

# This makes charts appear in the notebook
%matplotlib inline  

print("✅ Libraries imported successfully!")
print("📚 Ready to start analyzing close notes quality!")


## 2. Load Prepared Dataset

**What we're doing:** Loading the incident dataset from Notebook 01.

**Why:** We need all incidents with their close notes so we can identify which ones are high-quality and separate them.

**What to expect:** The dataset contains incidents with various quality levels of close notes. We'll filter to find the best ones.


In [ ]:
# Load prepared dataset from notebook 01
# This is the data we prepared in the previous notebook

data_dir = Path("../data")
prepared_path = data_dir / "incidents_prepared.csv"

if prepared_path.exists():
    # Load the prepared dataset (faster - we already processed it)
    df = pd.read_csv(prepared_path)
    print(f"✅ Loaded prepared dataset: {len(df)} records")
else:
    # If the prepared file doesn't exist, load from scratch
    print("⚠️ Prepared dataset not found. Loading from Hugging Face...")
    df = load_incident_dataset(sample_size=200, random_state=42)
    # Filter for records with close_notes (we need these!)
    if 'close_notes' in df.columns:
        df = df[df['close_notes'].notna()].copy()
    print(f"✅ Loaded dataset: {len(df)} records with close_notes")

# Let's see what we're working with
print(f"\n📊 Dataset Overview:")
print(f"   Total incidents: {len(df)}")
print(f"   Columns (pieces of information): {df.shape[1]}")
print(f"\n📋 Key columns we'll use:")
key_columns = ['close_notes', 'info_score_close_notes', 'info_score_poor_close_notes', 'category']
for col in key_columns:
    if col in df.columns:
        print(f"   ✅ {col}")
    else:
        print(f"   ❌ {col} (missing!)")


## 3. Understand Quality Scores

**What we're doing:** Looking at how close notes are scored for quality.

**What are quality scores?**
- **`info_score_close_notes`**: Measures how informative a close note is (0.0 to 1.0)
  - **High score (≥0.8)**: Contains detailed, useful information
  - **Low score (<0.8)**: Lacks detail or information
  
- **`info_score_poor_close_notes`**: Measures how "generic" a close note is (0.0 to 1.0)
  - **Low score (≤0.1)**: Not generic, contains specific information
  - **High score (>0.1)**: Generic phrases like "Issue resolved"

**Why this matters:** We want close notes that are informative (high score) and not generic (low poor score).

**What to look for:** 
- How many close notes have high information scores?
- How many have low generic scores?
- This tells us how many "good" examples we can use as references


In [ ]:
# Check quality score columns
# This shows us how the close notes are scored for quality

print("="*80)
print("📊 QUALITY SCORES ANALYSIS")
print("="*80)
print("\n💡 Understanding the scores:")
print("   • Higher info_score_close_notes = More informative (better!)")
print("   • Lower info_score_poor_close_notes = Less generic (better!)")
print("   • We want: High info score (≥0.8) AND Low poor score (≤0.1)")

if 'info_score_close_notes' in df.columns:
    high_quality_count = (df['info_score_close_notes'] >= 0.8).sum()
    high_quality_pct = (high_quality_count / len(df)) * 100
    print(f"\n📊 info_score_close_notes (How informative?):")
    print(f"   Mean (average): {df['info_score_close_notes'].mean():.3f}")
    print(f"   Median (middle value): {df['info_score_close_notes'].median():.3f}")
    print(f"   Range: {df['info_score_close_notes'].min():.3f} to {df['info_score_close_notes'].max():.3f}")
    print(f"   ✅ High quality (≥0.8): {high_quality_count} incidents ({high_quality_pct:.1f}%)")

if 'info_score_poor_close_notes' in df.columns:
    low_generic_count = (df['info_score_poor_close_notes'] <= 0.1).sum()
    low_generic_pct = (low_generic_count / len(df)) * 100
    print(f"\n📊 info_score_poor_close_notes (How generic?):")
    print(f"   Mean (average): {df['info_score_poor_close_notes'].mean():.3f}")
    print(f"   Median (middle value): {df['info_score_poor_close_notes'].median():.3f}")
    print(f"   ✅ Low generic (≤0.1): {low_generic_count} incidents ({low_generic_pct:.1f}%)")

# Visualize score distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if 'info_score_close_notes' in df.columns:
    axes[0].hist(df['info_score_close_notes'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0].axvline(0.8, color='red', linestyle='--', label='Threshold (≥0.8)')
    axes[0].set_xlabel('Info Score (close_notes)', fontsize=11)
    axes[0].set_ylabel('Frequency', fontsize=11)
    axes[0].set_title('Close Notes Quality Score Distribution', fontsize=12, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    axes[0].legend()

if 'info_score_poor_close_notes' in df.columns:
    axes[1].hist(df['info_score_poor_close_notes'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='coral')
    axes[1].axvline(0.1, color='red', linestyle='--', label='Threshold (≤0.1)')
    axes[1].set_xlabel('Info Score (poor_close_notes)', fontsize=11)
    axes[1].set_ylabel('Frequency', fontsize=11)
    axes[1].set_title('Poor Close Notes Score Distribution', fontsize=12, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    axes[1].legend()

plt.tight_layout()
plt.show()

print("="*80)


## 4. Define Quality Criteria and Filter

**What we're doing:** Applying filters to identify high-quality close notes that will become our "Reference Dataset".

**Our quality criteria:**
1. **High information score** (`info_score_close_notes` ≥ 0.8)
   - *Why?* Ensures the close note contains useful, detailed information
   
2. **Low generic score** (`info_score_poor_close_notes` ≤ 0.1)
   - *Why?* Ensures it's not just generic phrases like "Issue resolved"
   
3. **Not generic phrases** - Excludes notes that only say things like:
   - "No changes noted"
   - "Issue resolved"
   - "Resolved per user"
   
4. **Minimum length** (≥100 characters)
   - *Why?* Ensures there's enough detail to be useful

**What happens:**
- We'll filter the dataset step by step
- Records that pass all filters → **Reference Dataset** (high-quality examples)
- Records that don't pass → **Other Incidents Dataset** (for comparison)


In [ ]:
# Define generic phrases to exclude
# These are phrases that don't provide useful information
# Think of them like "filler words" - they don't tell us anything useful

GENERIC_PHRASES = [
    'no changes noted',
    'issue resolved',
    'resolved',
    'closed',
    'no further action',
    'resolved per user',
    'user confirmed resolved'
]

def is_generic_close_note(text):
    """
    Check if close note contains only generic phrases.
    
    This function looks for close notes that are too short or only contain
    generic phrases like "Issue resolved" without any details.
    """
    if pd.isna(text) or not isinstance(text, str):
        return True  # Missing or not text = consider it generic
    
    text_lower = text.lower().strip()
    
    # Check if text is too short (likely generic)
    # Good close notes need detail, so very short ones are probably generic
    if len(text_lower) < 50:
        return True
    
    # Check if text contains only generic phrases
    words = set(text_lower.split())
    generic_words = set()
    for phrase in GENERIC_PHRASES:
        generic_words.update(phrase.split())
    
    # If most words are generic, likely a generic note
    # (e.g., "Issue resolved" = only 2 words, both generic)
    if len(words) <= 5 and words.issubset(generic_words):
        return True
    
    return False  # Not generic - has useful information!

# Apply filters step by step
# We'll filter the data multiple times, keeping only the best close notes
print("🔍 Applying quality filters...")
print("="*80)
print(f"📦 Initial records: {len(df)}")

# Filter 1: Must have close_notes
# We can't evaluate quality if there's no close note!
df_filtered = df[df['close_notes'].notna()].copy()
print(f"✅ Filter 1 - Has close_notes: {len(df_filtered)} records")

# Filter 2: High quality score (informative)
# We want close notes that are informative (score ≥ 0.8)
if 'info_score_close_notes' in df_filtered.columns:
    before = len(df_filtered)
    df_filtered = df_filtered[df_filtered['info_score_close_notes'] >= 0.8].copy()
    removed = before - len(df_filtered)
    print(f"✅ Filter 2 - High info score (≥0.8): {len(df_filtered)} records (removed {removed})")

# Filter 3: Low poor quality score (not generic)
# We want close notes that are NOT generic (score ≤ 0.1)
if 'info_score_poor_close_notes' in df_filtered.columns:
    before = len(df_filtered)
    df_filtered = df_filtered[df_filtered['info_score_poor_close_notes'] <= 0.1].copy()
    removed = before - len(df_filtered)
    print(f"✅ Filter 3 - Low generic score (≤0.1): {len(df_filtered)} records (removed {removed})")

# Filter 4: Exclude generic notes (text-based check)
# Double-check: remove any that are just generic phrases
df_filtered['is_generic'] = df_filtered['close_notes'].apply(is_generic_close_note)
before = len(df_filtered)
df_filtered = df_filtered[~df_filtered['is_generic']].copy()
removed = before - len(df_filtered)
print(f"✅ Filter 4 - Exclude generic phrases: {len(df_filtered)} records (removed {removed})")

# Filter 5: Minimum text length (ensure informative)
# Very short close notes probably don't have enough detail
df_filtered['close_notes_length'] = df_filtered['close_notes'].astype(str).str.len()
before = len(df_filtered)
df_filtered = df_filtered[df_filtered['close_notes_length'] >= 100].copy()
removed = before - len(df_filtered)
print(f"✅ Filter 5 - Minimum length (≥100 chars): {len(df_filtered)} records (removed {removed})")

print("="*80)
print(f"\n🎯 Final filtered dataset: {len(df_filtered)} high-quality records")
reduction_pct = ((len(df) - len(df_filtered))/len(df)*100)
print(f"   📉 Filtered out: {len(df) - len(df_filtered)} records ({reduction_pct:.1f}%)")
print(f"   ✅ Kept: {len(df_filtered)} records ({100-reduction_pct:.1f}%)")
print("="*80)


## 5. Analyze Diversity of Filtered Dataset

**What we're doing:** Checking how diverse our filtered high-quality close notes are across different categories.

**Why this matters:**
- We want examples from different types of incidents (not just SOFTWARE)
- Diverse examples help when evaluating different categories later
- This shows us if we need to balance the dataset better

**What we'll check:**
- **Categories**: How many incidents per category (SOFTWARE, ACCOUNT, etc.)
- **Subcategories**: How many per subcategory (ERROR, MALFUNCTION, etc.)
- **Contact Types**: How incidents were reported (Email, Phone, Chat, etc.)

**What to look for:**
- If one category dominates (e.g., 90% SOFTWARE), we may want to balance
- Good diversity means we have examples across different incident types
- This helps when evaluating close notes for different types of problems


In [ ]:
# Analyze diversity
print("="*80)
print("DIVERSITY ANALYSIS")
print("="*80)

if 'category' in df_filtered.columns:
    print(f"\n📊 Categories:")
    category_counts = df_filtered['category'].value_counts()
    for cat, count in category_counts.items():
        print(f"   {cat}: {count} ({count/len(df_filtered)*100:.1f}%)")

if 'subcategory' in df_filtered.columns:
    print(f"\n📋 Subcategories:")
    subcat_counts = df_filtered['subcategory'].value_counts()
    print(f"   Total unique: {df_filtered['subcategory'].nunique()}")
    for subcat, count in subcat_counts.head(10).items():
        print(f"   {subcat}: {count}")

if 'contact_type' in df_filtered.columns:
    print(f"\n📞 Contact Types:")
    contact_counts = df_filtered['contact_type'].value_counts()
    for contact, count in contact_counts.items():
        print(f"   {contact}: {count} ({count/len(df_filtered)*100:.1f}%)")

# Visualize diversity
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

if 'category' in df_filtered.columns:
    category_counts = df_filtered['category'].value_counts()
    axes[0].bar(range(len(category_counts)), category_counts.values, 
                color=sns.color_palette("husl", len(category_counts)))
    axes[0].set_xticks(range(len(category_counts)))
    axes[0].set_xticklabels(category_counts.index, rotation=45, ha='right')
    axes[0].set_ylabel('Count', fontsize=10)
    axes[0].set_title('Category Distribution (Filtered)', fontsize=12, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(category_counts.values):
        axes[0].text(i, v, str(v), ha='center', va='bottom', fontsize=9)

if 'subcategory' in df_filtered.columns:
    subcat_counts = df_filtered['subcategory'].value_counts().head(10)
    axes[1].barh(range(len(subcat_counts)), subcat_counts.values,
                color=sns.color_palette("viridis", len(subcat_counts)))
    axes[1].set_yticks(range(len(subcat_counts)))
    axes[1].set_yticklabels(subcat_counts.index)
    axes[1].set_xlabel('Count', fontsize=10)
    axes[1].set_title('Top 10 Subcategories (Filtered)', fontsize=12, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)

if 'contact_type' in df_filtered.columns:
    contact_counts = df_filtered['contact_type'].value_counts()
    axes[2].bar(contact_counts.index, contact_counts.values,
               color=sns.color_palette("muted", len(contact_counts)))
    axes[2].set_ylabel('Count', fontsize=10)
    axes[2].set_title('Contact Type Distribution (Filtered)', fontsize=12, fontweight='bold')
    axes[2].grid(axis='y', alpha=0.3)
    for i, v in enumerate(contact_counts.values):
        axes[2].text(i, v, str(v), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("="*80)


## 6. Ensure Balanced Sampling

**What we're doing:** Making sure our Reference Dataset has examples from different categories.

**Why this matters:**
- We want diverse examples (not just SOFTWARE incidents)
- This helps when evaluating different types of incidents later
- Balanced sampling ensures we have good examples across categories

**How it works:**
- We try to get a target number of samples per category (e.g., 20)
- If a category has fewer samples, we take what's available
- If a category has too few samples (< 5), we skip it

**Result:** A balanced Reference Dataset with diverse examples.

**Note:** After sampling, the remaining high-quality records (that didn't make it into the balanced sample) will still be part of the "Other Incidents Dataset" - they're still good quality, just not selected for the reference set.


In [ ]:
# Strategy: Sample balanced examples if one category dominates
TARGET_SAMPLES_PER_CATEGORY = 20  # Adjust based on dataset size
MIN_SAMPLES_PER_CATEGORY = 5     # Minimum samples per category

print("Applying balanced sampling strategy...")

if 'category' in df_filtered.columns:
    sampled_records = []
    
    for category in df_filtered['category'].unique():
        category_df = df_filtered[df_filtered['category'] == category].copy()
        
        if len(category_df) >= TARGET_SAMPLES_PER_CATEGORY:
            # Sample TARGET_SAMPLES_PER_CATEGORY records
            sampled = category_df.sample(
                min(TARGET_SAMPLES_PER_CATEGORY, len(category_df)),
                random_state=42
            )
        elif len(category_df) >= MIN_SAMPLES_PER_CATEGORY:
            # Take all available if less than target but above minimum
            sampled = category_df
        else:
            # Skip categories with too few samples
            print(f"   ⚠️ Skipping {category}: only {len(category_df)} samples")
            continue
        
        sampled_records.append(sampled)
        print(f"   ✅ {category}: {len(sampled)} samples")
    
    df_ground_truth = pd.concat(sampled_records, ignore_index=True)
    print(f"\n✅ Balanced ground truth dataset: {len(df_ground_truth)} records")
else:
    # If no category column, use all filtered records
    df_ground_truth = df_filtered.copy()
    print(f"\n✅ Using all filtered records: {len(df_ground_truth)} records")


## 7. Create Reference Dataset Structure

**What we're doing:** Preparing the Reference Dataset with the columns we need.

**Structure we need:**
- `number` - Incident identifier
- `content` - Original incident description
- `close_notes_ref` - High-quality close note (reference example)
- Metadata: category, subcategory, contact_type, info_score

**Why these columns:**
- We need both `content` and `close_notes_ref` from the same incident (for Notebook 03)
- Metadata helps us find similar incidents for evaluation


In [ ]:
# Create ground truth dataset with required structure
gt_dataset = pd.DataFrame({
    'number': df_ground_truth['number'].values,
    'content': df_ground_truth['content'].values,
    'close_notes_ref': df_ground_truth['close_notes'].values
})

# Add optional metadata columns for reference
if 'category' in df_ground_truth.columns:
    gt_dataset['category'] = df_ground_truth['category'].values
if 'subcategory' in df_ground_truth.columns:
    gt_dataset['subcategory'] = df_ground_truth['subcategory'].values
if 'contact_type' in df_ground_truth.columns:
    gt_dataset['contact_type'] = df_ground_truth['contact_type'].values
if 'info_score_close_notes' in df_ground_truth.columns:
    gt_dataset['info_score'] = df_ground_truth['info_score_close_notes'].values

print("="*80)
print("GROUND TRUTH DATASET STRUCTURE")
print("="*80)
print(f"\nTotal records: {len(gt_dataset)}")
print(f"\nColumns: {list(gt_dataset.columns)}")
print(f"\nFirst few records:")
print(gt_dataset.head())
print("="*80)


## 8. Separate into Two Datasets

**What we're doing:** Splitting our incidents into two groups - the "good examples" and everything else.

**Why separate:**
- **Reference Dataset** (the "good" ones): High-quality close notes that will serve as our standards
  - Think of these like "model answers" - examples of what good close notes should look like
  - We'll use these to evaluate other close notes (existing or AI-generated)
  
- **Other Incidents Dataset** (the rest): All remaining incidents that didn't meet our high-quality criteria
  - These are standard close notes - not bad, just not exceptional
  - We'll compare these against the reference to see the difference

**How we separate:**
- Records that passed all quality filters → **Reference Dataset** ✅
- All other records → **Other Incidents Dataset** 📋

**Think of it like:**
- Sorting apples into "premium" (perfect, beautiful) and "regular" (good, but not perfect)
- The premium ones become our reference for what "good" looks like
- The regular ones help us see the difference

**This separation allows us to:**
- Compare how "good" close notes differ from standard ones
- Use references to evaluate other close notes (like grading against a rubric)
- Understand quality differences between datasets


In [ ]:
# Separate into two datasets
# This is the key step: creating Reference Dataset and Other Incidents Dataset

print("="*80)
print("DATASET SEPARATION")
print("="*80)

# Reference Dataset: High-quality close notes (what we've filtered)
# Contains: number, content, close_notes_ref, and metadata
reference_dataset = gt_dataset.copy()

# Other Incidents Dataset: All incidents NOT in reference dataset
# These are incidents that didn't meet the high-quality criteria
reference_numbers = set(reference_dataset['number'].values)
other_incidents = df[~df['number'].isin(reference_numbers)].copy()

# Keep same structure for other incidents (for consistency)
# Keep: number, content, close_notes (if available), and metadata
other_incidents_dataset = other_incidents[[
    'number', 'content', 'close_notes', 'category', 'subcategory', 'contact_type'
]].copy() if 'close_notes' in other_incidents.columns else other_incidents[['number', 'content', 'category', 'subcategory', 'contact_type']].copy()

print(f"\n📊 Separation Summary:")
print(f"   Total incidents: {len(df)}")
print(f"   ────────────────────────────────────────")
print(f"   ✅ Reference Dataset (high-quality): {len(reference_dataset)} incidents")
print(f"      - These are our 'good' examples")
print(f"      - Will be used as references for evaluation")
print(f"   ────────────────────────────────────────")
print(f"   📋 Other Incidents Dataset: {len(other_incidents_dataset)} incidents")
print(f"      - Remaining incidents (standard quality)")
print(f"      - Will be used for comparison")
print(f"   ────────────────────────────────────────")
print(f"   Total: {len(reference_dataset) + len(other_incidents_dataset)} incidents")
print(f"   Match check: {'✅ Match' if len(reference_dataset) + len(other_incidents_dataset) == len(df) else '⚠️ Mismatch'}")

# Show examples of the difference
print(f"\n📝 Example from Reference Dataset (High-Quality):")
if len(reference_dataset) > 0:
    example_ref = reference_dataset.iloc[0]
    print(f"   Incident: {example_ref['number']}")
    print(f"   Close Note: {str(example_ref['close_notes_ref'])[:200]}...")
    print(f"   Quality Score: {example_ref.get('info_score', 'N/A')}")

print(f"\n📝 Example from Other Incidents Dataset (Standard):")
if len(other_incidents_dataset) > 0 and 'close_notes' in other_incidents_dataset.columns:
    example_other = other_incidents_dataset[other_incidents_dataset['close_notes'].notna()].iloc[0] if len(other_incidents_dataset[other_incidents_dataset['close_notes'].notna()]) > 0 else other_incidents_dataset.iloc[0]
    print(f"   Incident: {example_other['number']}")
    if 'close_notes' in example_other and pd.notna(example_other['close_notes']):
        print(f"   Close Note: {str(example_other['close_notes'])[:200]}...")
    else:
        print(f"   Close Note: (not available)")

print("\n💡 Key Difference:")
print("   - Reference Dataset: Detailed, informative, complete close notes")
print("   - Other Incidents: Standard close notes (may be shorter, less detailed, or generic)")

print("="*80)


## 9. Display Sample Examples

**What we're doing:** Showing examples from both datasets so you can see the difference.

**What to look for:**
- Reference examples are detailed and informative
- Other incidents may be shorter or less detailed
- This visual comparison helps understand what makes a "good" close note


In [ ]:
# Display sample examples
print("="*80)
print("SAMPLE GROUND TRUTH EXAMPLES")
print("="*80)

for idx, row in gt_dataset.head(3).iterrows():
    print(f"\n{'='*80}")
    print(f"Example {idx + 1}")
    print(f"{'='*80}")
    print(f"\n📋 Incident Number: {row['number']}")
    if 'category' in row:
        print(f"🏷️  Category: {row['category']}")
    if 'subcategory' in row:
        print(f"🏷️  Subcategory: {row['subcategory']}")
    if 'info_score' in row:
        print(f"⭐ Quality Score: {row['info_score']:.2f}")
    
    print(f"\n📝 Original Content (Input):")
    print(f"   {row['content'][:300]}...")
    
    print(f"\n✅ Reference Close Notes (Ground Truth):")
    print(f"   {row['close_notes_ref'][:400]}...")

print(f"\n{'='*80}")


## 10. Save Both Datasets

**What we're doing:** Saving both datasets so we can use them in later notebooks.

**What gets saved:**
1. **Reference Dataset** (`reference_close_notes.csv`) - **Good Samples**
   - High-quality close notes with both `content` and `close_notes_ref`
   - Used as references for evaluation
   
2. **Other Incidents Dataset** (`other_incidents.csv`) - **Remaining Samples**
   - All remaining incidents (those that didn't meet high-quality criteria)
   - Includes `content` and `close_notes` (if available)
   - Can be used for comparison in Notebook 03

**Why save both:**
- Reference Dataset: Used in Notebooks 03, 04, and 05 for evaluation
- Other Incidents Dataset: Used for comparison to see quality differences
- Saving both allows us to work with consistent data across notebooks
- Both files are ready to use without re-running this notebook


In [ ]:
# Save Reference Dataset (high-quality close notes)
# This is the main dataset we'll use as references for evaluation

reference_final = reference_dataset[[
    'number',
    'content',
    'close_notes_ref'
]].copy()

# Add metadata columns if they exist
for col in ['category', 'subcategory', 'contact_type', 'info_score']:
    if col in reference_dataset.columns:
        reference_final[col] = reference_dataset[col]

# Save Reference Dataset (good samples)
reference_path = data_dir / "reference_close_notes.csv"
reference_final.to_csv(reference_path, index=False)

# Prepare Other Incidents Dataset for saving
# This dataset already has the structure we need: number, content, close_notes, and metadata
other_incidents_final = other_incidents_dataset.copy()

# Save Other Incidents Dataset (remaining incidents - not high-quality)
other_incidents_path = data_dir / "other_incidents.csv"
other_incidents_final.to_csv(other_incidents_path, index=False)

print("="*80)
print("DATASETS SAVED")
print("="*80)

print(f"\n✅ Reference Dataset (Good Samples) saved:")
print(f"   File: {reference_path}")
print(f"   Total records: {len(reference_final)}")
print(f"   File size: {reference_path.stat().st_size / 1024:.1f} KB")
print(f"   Columns: {list(reference_final.columns)}")
print(f"   Use: High-quality examples for evaluation (Notebooks 03, 04, 05)")

print(f"\n✅ Other Incidents Dataset (Remaining Samples) saved:")
print(f"   File: {other_incidents_path}")
print(f"   Total records: {len(other_incidents_final)}")
print(f"   File size: {other_incidents_path.stat().st_size / 1024:.1f} KB")
print(f"   Columns: {list(other_incidents_final.columns)}")
print(f"   Use: For comparison in Notebook 03")

print("\n💡 Summary:")
print(f"   - Reference Dataset (good): {len(reference_final)} high-quality examples → {reference_path.name}")
print(f"   - Other Incidents (remaining): {len(other_incidents_final)} remaining incidents → {other_incidents_path.name}")
print(f"   - Total: {len(reference_final) + len(other_incidents_final)} incidents")
print("="*80)


## 11. Summary Statistics

**What we're doing:** Looking at the final results - what we created and how the two datasets compare.

**What we'll show:**
- **Size of each dataset** - How many incidents in each group?
- **Distribution across categories** - Do we have examples from different types of problems?
- **Quality differences** - How do the scores compare between "good" and "regular"?

**This helps us understand:**
- How many reference examples we have (enough to evaluate with?)
- Whether we have good coverage across categories (not just SOFTWARE?)
- The difference between reference and other incidents (is the quality gap clear?)

**Think of it like:** A final report card showing what we accomplished and what we have to work with.


In [ ]:
# Final summary comparing both datasets
# This shows us what we accomplished and what we have to work with

print("="*80)
print("🎯 FINAL SUMMARY: DATASET SEPARATION COMPLETE")
print("="*80)

print(f"\n✅ Reference Dataset (High-Quality Examples):")
print(f"   📦 Total records: {len(reference_final)}")
print(f"   🎯 Purpose: Examples of good close notes for evaluation")
print(f"   💡 Think of these as 'model answers' - what good close notes should look like")

if 'category' in reference_final.columns:
    print(f"\n   📊 Category Distribution:")
    for cat, count in reference_final['category'].value_counts().items():
        pct = count/len(reference_final)*100
        print(f"      • {cat}: {count} incidents ({pct:.1f}%)")

if 'info_score' in reference_final.columns:
    print(f"\n   ⭐ Quality Scores:")
    print(f"      Mean (average): {reference_final['info_score'].mean():.3f}")
    print(f"      Range: {reference_final['info_score'].min():.3f} to {reference_final['info_score'].max():.3f}")
    print(f"      💡 All scores are ≥ 0.8 (high quality!)")

print(f"\n📋 Other Incidents Dataset (Remaining Samples):")
print(f"   📦 Total records: {len(other_incidents_dataset)}")
print(f"   🎯 Purpose: Remaining incidents for comparison")
print(f"   💡 These are standard close notes - not bad, just not exceptional")

if 'category' in other_incidents_dataset.columns:
    print(f"\n   📊 Category Distribution:")
    for cat, count in other_incidents_dataset['category'].value_counts().items():
        pct = count/len(other_incidents_dataset)*100
        print(f"      • {cat}: {count} incidents ({pct:.1f}%)")

print(f"\n📊 Overall Statistics:")
print(f"   📦 Total incidents: {len(df)}")
ref_pct = len(reference_final)/len(df)*100
other_pct = len(other_incidents_dataset)/len(df)*100
print(f"   ✅ Reference Dataset: {len(reference_final)} incidents ({ref_pct:.1f}%)")
print(f"   📋 Other Incidents: {len(other_incidents_dataset)} incidents ({other_pct:.1f}%)")

print(f"\n💾 Files Saved:")
print(f"   ✅ Reference Dataset: data/reference_close_notes.csv")
print(f"   ✅ Other Incidents: data/other_incidents.csv")

print(f"\n🚀 Next Steps:")
print(f"   → Notebook 03: Compare n-gram scores between reference and other incidents")
print(f"   → Notebook 04: Analyze semantic similarity using embeddings")
print(f"   → Notebook 05: Use LLM-as-a-Judge to evaluate close notes")
print("="*80)
